In [1]:
#FOR COLAB:
# import os
# from google.colab import drive

# # Mount drive and change directory
# drive.mount('/content/drive')

# path = 'NLP Project'
# os.chdir(f'/content/drive/MyDrive/{path}')
# agent_file = 'gridspace-stanford-harper-valley/data/audio/agent/0a6f57765f4641a2.wav'
# caller_file = 'gridspace-stanford-harper-valley/data/audio/caller/0a6f57765f4641a2.wav'

Mounted at /content/drive


In [17]:
#LOCAL:
import os 
#get current directory
path = os.getcwd()
#change to parent directory
os.chdir(os.path.dirname(path))
print(path)
agent_file = 'gridspace_repo/data/audio/agent/0a6f57765f4641a2.wav'
caller_file = 'gridspace_repo/data/audio/caller/0a6f57765f4641a2.wav'

/home/nicco/NLP/project/notebooks


In [2]:
# Install needed packages
!pip install SpeechRecognition
!pip install pydub

## Transcribe the audio files

In [8]:
!pip install pydub
!pip install speechrecognition
!pip install numpy
!pip install matplotlib
!pip install itertools

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached matplotlib-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached fonttools-4.51.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached kiwisolver-1.4.5-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached pillow-10.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
  Using cached contourpy-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (305 kB)
ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


In [9]:
from pydub import AudioSegment, silence
import speech_recognition as sr
import numpy as np
from itertools import zip_longest

/home/nicco/NLP/nlpvenv/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Split in terms of silence pauses

In [21]:
def split_audio_on_silence(audio_path, min_silence_len=2500):
    """Load an audio file and split on silences."""
    audio = AudioSegment.from_wav(audio_path)
    # This may need adjustment based on the specific audio file characteristics
    return silence.split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=audio.dBFS-15, keep_silence=100)

def transcribe_segments(segments):
    """Transcribe a list of AudioSegment objects."""
    recognizer = sr.Recognizer()
    transcriptions = []

    for i, segment in enumerate(segments):
        # Export segment to a temporary WAV file
        segment.export(f"temp_segment{i}.wav", format="wav")

        # Recognize the audio
        with sr.AudioFile("temp_segment.wav") as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio_data = recognizer.record(source)
            try:
                transcription = recognizer.recognize_google(audio_data, language='en-US', show_all=False)
                transcriptions.append(transcription)
            except sr.UnknownValueError:
                print(f"Segment {i+1}: Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Segment {i+1}: Could not request results; {e}")

    return transcriptions

def main(audio_file):
    segments = split_audio_on_silence(audio_file)
    print(segments)
    transcriptions = transcribe_segments(segments)
    return transcriptions

In [22]:
# Paths to files
agent_transcription = main(agent_file)
caller_transcription = main(caller_file)

print('Conversation\n')
for agent, caller in zip_longest(agent_transcription, caller_transcription, fillvalue="???"):
    print("count \r")
    if agent:
        print(f'Agent: {agent}')
    if caller:
        print(f'Caller: {caller}')

[<pydub.audio_segment.AudioSegment object at 0x7fb222c9eb30>, <pydub.audio_segment.AudioSegment object at 0x7fb2201a9bd0>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee645330>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee645c90>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee645300>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee645210>]
Segment 1: Google Speech Recognition could not understand audio
Segment 2: Google Speech Recognition could not understand audio
Segment 3: Google Speech Recognition could not understand audio
Segment 4: Google Speech Recognition could not understand audio
Segment 5: Google Speech Recognition could not understand audio
Segment 6: Google Speech Recognition could not understand audio
[<pydub.audio_segment.AudioSegment object at 0x7fb1ee645210>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee644ac0>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee644880>, <pydub.audio_segment.AudioSegment object at 0x7fb1ee644a90>, <pyd

## Split following the transcripts (just caller for now)

have to look at the readme file to understand which fields are crucial

In [43]:
import json
import speech_recognition as sr
from pydub import AudioSegment
from difflib import SequenceMatcher

def load_transcript(file_path):
    with open(file_path, 'r') as f:
        transcript = json.load(f)
    return transcript

def split_audio(file_path, transcript):
    audio = AudioSegment.from_wav(file_path)
    segments = []
    for segment in transcript:
        start = segment['start_ms'] #when the segment start (offset from the beginning of the audio file in milliseconds)
        duration = segment['duration_ms'] #how long the segment is (in milliseconds)
        segments.append(audio[start:start+duration]) 
    return segments

def transcribe_segments(segments):
    r = sr.Recognizer()
    transcriptions = []
    for i, segment in enumerate(segments):
        #create a temp folder to store the audio files
        os.makedirs("temp", exist_ok=True)
        segment.export(f"temp/temp_segment{i}.wav", format="wav")
        with sr.AudioFile(f"temp/temp_segment{i}.wav") as source:
            audio = r.record(source)
            try:
                transcription = r.recognize_google(audio)
                transcriptions.append(transcription)
            except sr.UnknownValueError: # Google Speech Recognition could not understand the audio, catch this exception
                transcriptions.append("did not understand:(")
                print(f"Google Speech Recognition could not understand audio from segment {i}")
    return transcriptions

def compare_transcriptions(transcriptions, transcript):
    for i, transcription in enumerate(transcriptions):
        human_transcript = transcript[i]['human_transcript']
        similarity = SequenceMatcher(None, transcription, human_transcript).ratio()
        print(f"Segment {i}:")
        print(f"Machine transcript: {transcription}")
        print(f"Human transcript: {human_transcript}")
        print(f"Similarity: {similarity}")

def main(audio_file_path, transcript_file_path):
    transcript = load_transcript(transcript_file_path)
    segments = split_audio(audio_file_path, transcript)
    transcriptions = transcribe_segments(segments)
    compare_transcriptions(transcriptions, transcript)

#
main('gridspace_repo/data/audio/caller/0a6f57765f4641a2.wav', 'gridspace_repo/data/transcript/0a6f57765f4641a2.json')

Google Speech Recognition could not understand audio from segment 0
Google Speech Recognition could not understand audio from segment 3
Google Speech Recognition could not understand audio from segment 5
Google Speech Recognition could not understand audio from segment 6
Google Speech Recognition could not understand audio from segment 8
Google Speech Recognition could not understand audio from segment 9
Google Speech Recognition could not understand audio from segment 10
Google Speech Recognition could not understand audio from segment 11
Google Speech Recognition could not understand audio from segment 12
Google Speech Recognition could not understand audio from segment 13
Google Speech Recognition could not understand audio from segment 15
Google Speech Recognition could not understand audio from segment 16
Google Speech Recognition could not understand audio from segment 17
Google Speech Recognition could not understand audio from segment 18
Google Speech Recognition could not unde

In [42]:
#delete the temp folder
import shutil
shutil.rmtree("temp")

About 70% of the segments cannot be understood by this model, we will have to fine tune a new one.